In [ ]:
# For developers:
#using Pkg
#Pkg.activate("../FUSE/")
using Plots # for plotting
using FUSE # this will also import IMAS in the current namespace
using TGLFNN

The FUSE flux-matcher has been developed at General Atomics with algorithms based on the TGYRO flux-matcher and an optimizer that is more advanced.

In [ ]:
# Flux-match the DIII-D standard FUSE L-mode case
ini, act = FUSE.case_parameters(:D3D, :L_mode);      # This line can take 15 minutes to run when run for the first time in your project (compilation time)
dd = IMAS.dd()
FUSE.init(dd, ini, act);                         
act.ActorCoreTransport.model = :FluxMatcher;
act.ActorTGLF.warn_nn_train_bounds = false           # print warnings about leaving NN training bounds
act.ActorTGLF.model = :TGLFNN                        # can be :TGLFNN, :TJLF (Julia version of Fortran TGLF), :TGLF (needs GACODE insalled), :QLGYRO (needs GACODE installed)
act.ActorTGLF.onnx_model = false                     # you can run your own PyTorch-trained model saved in ONNX format and placed in TGLFNN repo here: /home/<user>/.julia/dev/TGLFNN/models/
act.ActorTGLF.tglfnn_model = "sat1_em_d3d_azf-1"     # NN model can be symbol (e.g. :sat1), or string of model name (e.g. "sat2_em_d3d_azf-1")
act.ActorTGLF.electromagnetic = true                 # this setting is not relevant for NNs, which have fixed settings from training database

act.ActorFluxMatcher.rho_transport = 0.1:0.05:0.85
act.ActorFluxMatcher.max_iterations = 500
act.ActorFluxMatcher.optimizer_algorithm = :anderson #:simple (best only if monotonically increasing profiles), :anderson, or :jacobian_based
act.ActorFluxMatcher.step_size = 0.5
act.ActorFluxMatcher.verbose = true
act.ActorFluxMatcher.evolve_rotation = :flux_match   # :flux_match or :fixed
act.ActorFluxMatcher.evolve_pedestal = false         # take experimental pedestal
act.ActorFluxMatcher.evolve_densities = :flux_match  # :flux_match or :fixed

# Show profiles before flux-matcher run
display(act.ActorFluxMatcher)
dd_old = deepcopy(dd)

#FUSE.ActorPedestal(dd,act)
actor_transport = FUSE.ActorFluxMatcher(dd, act)

# show after
display(plot(dd_old.core_profiles, label=" Experiment"))
display(plot!(dd.core_profiles, label=" TGLF-NN"))

# plot the flux_matching
display(plot(dd.core_transport))

In [ ]:
# Flux-match the DIII-D standard FUSE H-mode case
ini, act = FUSE.case_parameters(:D3D, :H_mode);        # This line can take 15 minutes to run when run for the first time in your project (compilation time)
dd = IMAS.dd()
FUSE.init(dd, ini, act);                         
act.ActorCoreTransport.model = :FluxMatcher;
act.ActorTGLF.warn_nn_train_bounds = false             # print warnings about leaving NN training bounds
act.ActorTGLF.model = :TGLFNN                          # can be :TGLFNN, :TJLF (Julia version of Fortran TGLF), :TGLF (needs GACODE insalled), :QLGYRO (needs GACODE installed)
act.ActorTGLF.onnx_model = false                       # you can run your own PyTorch-trained model saved in ONNX format and placed in TGLFNN repo here: /home/<user>/.julia/dev/TGLFNN/models/
act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"       # NN model can be symbol (e.g. :sat1), or string of model name (e.g. "sat2_em_d3d_azf-1")
act.ActorTGLF.electromagnetic = true                   # this setting is not relevant for NNs, which have fixed settings from training database

act.ActorFluxMatcher.rho_transport = 0.1:0.05:0.85
act.ActorFluxMatcher.max_iterations = 1000
act.ActorFluxMatcher.optimizer_algorithm = :anderson   #:simple (best only if monotonically increasing profiles), :anderson, or :jacobian_based
act.ActorFluxMatcher.step_size = 1.0
act.ActorFluxMatcher.verbose = true
act.ActorFluxMatcher.evolve_rotation = :flux_match     # :flux_match or :fixed
act.ActorFluxMatcher.evolve_pedestal = false           # take experimental pedestal
act.ActorFluxMatcher.evolve_densities = :flux_match    # :flux_match or :fixed

# Show profiles before flux-matcher run
display(act.ActorFluxMatcher)
dd_old = deepcopy(dd)

#FUSE.ActorPedestal(dd,act)
actor_transport = FUSE.ActorFluxMatcher(dd, act)

# show after
display(plot(dd_old.core_profiles, label=" Experiment"))
display(plot!(dd.core_profiles, label=" TGLF-NN"))

# plot the flux_matching
display(plot(dd.core_transport))